In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

input_ = pd.read_excel('./part_number.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[['JOIN_MPN', 'Part_Type', 'Part_Key']]
input_.drop_duplicates(['JOIN_MPN'],
                       keep='first',
                       ignore_index=True)

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

import numpy as np

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                jsn = '{"partData":{"listing_data_essential":{"parttype":"' + input_.loc[i, 'Part_Type'] + '","partkey":"' + input_.loc[i, 'Part_Key'] + '"}}}'

                payload = {'func': 'getbuyersguide',
                           'payload': jsn,
                           'api_json_request': '1'}
                
                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=payload,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text
                
                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['buyersguidepieces']['body']

                soup = BeautifulSoup(html, 'lxml')

                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))
                
                security_code = html.xpath('//div[@class="center"]/div')
                
                # = = = = = = = = = = = = = = =
                
                if len(security_code) != 0 and 'security code' in security_code[0].text:
                    continue
                
                # = = = = = = = = = = = = = = =

                list_vehicle = []
                list_tr = html.xpath('//div[contains(@class, "buyersguide-nested")]/div/table[contains(@class, "nobmp")]/tr')
                for tr in list_tr:
                    list_td = [text.strip() for text in tr.xpath('./td/text()')]
                    list_vehicle.append(' '.join(list_td).strip())

                vehicle = '\n'.join(list_vehicle)

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'JOIN_MPN': input_.loc[i, 'JOIN_MPN'],
                                         'Vehicle': vehicle}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'

                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status != 'ok':
            df_temp = pd.DataFrame([{'JOIN_MPN': input_.loc[i, 'JOIN_MPN'],
                                     'Part_Type': input_.loc[i, 'Part_Type'],
                                     'Part_Key': input_.loc[i, 'Part_Key']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Part_Type'] + ' - ' + input_.loc[i, 'Part_Key'] + '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['JOIN_MPN'], ascending=[True])
output_correct.to_excel('./vehicle-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['JOIN_MPN'], ascending=[True])
    output_error.to_excel('./vehicle_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：475

5756 - 8304984  <->  [ok] - 剩余数量：455 - 当前时间：13:51:13
5756 - 15382993  <->  [ok] - 剩余数量：454 - 当前时间：13:51:13
5756 - 8304980  <->  [ok] - 剩余数量：453 - 当前时间：13:51:13
5756 - 11158045  <->  [ok] - 剩余数量：452 - 当前时间：13:51:13
5756 - 8304976  <->  [ok] - 剩余数量：451 - 当前时间：13:51:13
5756 - 8304992  <->  [ok] - 剩余数量：450 - 当前时间：13:51:13
5756 - 10454500  <->  [ok] - 剩余数量：449 - 当前时间：13:51:13
5756 - 8304972  <->  [ok] - 剩余数量：448 - 当前时间：13:51:13
5756 - 10454504  <->  [ok] - 剩余数量：447 - 当前时间：13:51:14
5756 - 9658988  <->  [ok] - 剩余数量：446 - 当前时间：13:51:14
5756 - 13570025  <->  [ok] - 剩余数量：445 - 当前时间：13:51:14
5756 - 8305012  <->  [ok] - 剩余数量：444 - 当前时间：13:51:14
5756 - 8921200  <->  [ok] - 剩余数量：443 - 当前时间：13:51:14
5756 - 8921212  <->  [ok] - 剩余数量：442 - 当前时间：13:51:14
5756 - 8305016  <->  [ok] - 剩余数量：441 - 当前时间：13:51:14
5756 - 8921208  <->  [ok] - 剩余数量：440 - 当前时间：13:51:14
5756 - 9722076  <->  [ok] - 剩余数量：439 - 当前时间：13:51:14
5756 - 13570029  <->  [ok] - 剩余数量：438 - 当前时间：13:51:14
5756 - 15382989  <->  [ok] - 剩余

5756 - 8304968  <->  [ok] - 剩余数量：302 - 当前时间：13:51:48
5756 - 8305264  <->  [ok] - 剩余数量：301 - 当前时间：13:51:48
5756 - 8305240  <->  [ok] - 剩余数量：300 - 当前时间：13:51:48
5756 - 8305156  <->  [ok] - 剩余数量：299 - 当前时间：13:51:48
5756 - 8305192  <->  [ok] - 剩余数量：298 - 当前时间：13:51:49
5756 - 8305292  <->  [ok] - 剩余数量：297 - 当前时间：13:51:49
5756 - 8305276  <->  [ok] - 剩余数量：296 - 当前时间：13:51:49
5756 - 8305288  <->  [ok] - 剩余数量：295 - 当前时间：13:51:50
5756 - 8305316  <->  [ok] - 剩余数量：294 - 当前时间：13:51:52
5756 - 8305296  <->  [ok] - 剩余数量：293 - 当前时间：13:51:52
5756 - 14310125  <->  [ok] - 剩余数量：292 - 当前时间：13:51:52
5756 - 8305216  <->  [ok] - 剩余数量：291 - 当前时间：13:51:53
5756 - 14310121  <->  [ok] - 剩余数量：290 - 当前时间：13:51:53
5756 - 8305284  <->  [ok] - 剩余数量：289 - 当前时间：13:51:53
5756 - 11158133  <->  [ok] - 剩余数量：288 - 当前时间：13:51:54
5756 - 5031150  <->  [ok] - 剩余数量：287 - 当前时间：13:51:54
5756 - 11158129  <->  [ok] - 剩余数量：286 - 当前时间：13:51:54
5756 - 8305320  <->  [ok] - 剩余数量：285 - 当前时间：13:51:54
5756 - 8305272  <->  [ok] - 剩余数量：284 - 当前时

5756 - 8305636  <->  [ok] - 剩余数量：148 - 当前时间：13:52:38
5756 - 8305596  <->  [ok] - 剩余数量：147 - 当前时间：13:52:38
5756 - 13570069  <->  [ok] - 剩余数量：146 - 当前时间：13:52:39
5756 - 8305648  <->  [ok] - 剩余数量：145 - 当前时间：13:52:39
5756 - 6352956  <->  [ok] - 剩余数量：144 - 当前时间：13:52:39
5756 - 8305644  <->  [ok] - 剩余数量：143 - 当前时间：13:52:40
5756 - 13570065  <->  [ok] - 剩余数量：142 - 当前时间：13:52:40
5756 - 8305632  <->  [ok] - 剩余数量：141 - 当前时间：13:52:41
5756 - 8305668  <->  [ok] - 剩余数量：140 - 当前时间：13:52:41
5756 - 8305676  <->  [ok] - 剩余数量：139 - 当前时间：13:52:41
5756 - 8305672  <->  [ok] - 剩余数量：138 - 当前时间：13:52:41
5756 - 14912061  <->  [ok] - 剩余数量：137 - 当前时间：13:52:41
5756 - 9658996  <->  [ok] - 剩余数量：136 - 当前时间：13:52:41
5756 - 8305656  <->  [ok] - 剩余数量：135 - 当前时间：13:52:42
5756 - 8305744  <->  [ok] - 剩余数量：134 - 当前时间：13:52:42
5756 - 8305664  <->  [ok] - 剩余数量：133 - 当前时间：13:52:42
5756 - 10110004  <->  [ok] - 剩余数量：132 - 当前时间：13:52:42
5756 - 8305740  <->  [ok] - 剩余数量：131 - 当前时间：13:52:42
5756 - 8305756  <->  [ok] - 剩余数量：130 - 当前时

5756 - 8716112  <->  [ok] - 剩余数量：0 - 当前时间：13:53:23
5756 - 13570153  <->  [ok] - 剩余数量：0 - 当前时间：13:53:24
5756 - 13570165  <->  [ok] - 剩余数量：0 - 当前时间：13:53:24
5756 - 8716012  <->  [ok] - 剩余数量：0 - 当前时间：13:53:25
5756 - 14047985  <->  [ok] - 剩余数量：0 - 当前时间：13:53:27
5756 - 10454520  <->  [ok] - 剩余数量：0 - 当前时间：13:53:30
5756 - 8921204  <->  [ok] - 剩余数量：0 - 当前时间：13:54:17
5756 - 8305404  <->  [ok] - 剩余数量：0 - 当前时间：13:54:22
5756 - 8305544  <->  [ok] - 剩余数量：0 - 当前时间：13:55:40
5756 - 8305484  <->  [ok] - 剩余数量：0 - 当前时间：13:55:51
5756 - 13060781  <->  [ok] - 剩余数量：0 - 当前时间：13:56:14
5756 - 14047989  <->  [ok] - 剩余数量：0 - 当前时间：13:56:20

爬虫输出中

爬虫结束
